Initially we had created images without caring about the colormap of each image<br>
And then we identified which matFile consisted of invalid data.<br>
This included the automated and manual check we had to do on the images. <br><br>

To reduce the computation of checking the criteria of valid or invalid data.<br>
We will iterate through the csv files which is collection of valid images<br>
And create a json of .mat files consisting of valid frequencies.

In [1]:
import re
import json
import math
import scipy.io
import numpy as np
import pandas as pd

In [13]:
imgFreq = {}
tbsVal = {}
fList = ['19H','19V','19','22V','37V','37H','37','91H','91V','91','150H','183_1H','183_3H','183_7H']
rList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
for r in rList:
    tbsVal[r] = {}
    imgFreq[r] = {}
    for f in fList:
        tbsVal[r][f] = [math.inf,-math.inf]
        imgFreq[r][f] = pd.read_csv("..\\ImagesPerFreq\\"+r+"_"+f+'.csv')

### Code to find out Max Min Brightness temperatures

In [14]:
valid_matfile_and_best_track = json.load( open( "..\\valid_matfile_and_best_track.json" ) )

In [17]:
class FindMaxMinTbs:
    global tbsVal
    def __init__(self, matpath):
        self.mMatPath = matpath
        
        self.__mFreq_to_swath = {}
        self.__mFreq_to_swath['19']  = [0,[2.400, 1.400]] 
        self.__mFreq_to_swath['19V'] = [0,0]
        self.__mFreq_to_swath['19H'] = [0,1]
        self.__mFreq_to_swath['22V'] = [0,2]

        self.__mFreq_to_swath['37']  = [1,[2.150, 1.150]]
        self.__mFreq_to_swath['37V'] = [1,0]
        self.__mFreq_to_swath['37H'] = [1,1]

        self.__mFreq_to_swath['91']  = [3,[1.751, 0.751]]
        self.__mFreq_to_swath['91V'] = [3,0] 
        self.__mFreq_to_swath['91H'] = [3,1]

        self.__mFreq_to_swath['150H'] = [2,0]
        self.__mFreq_to_swath['183_1H'] = [2,1]
        self.__mFreq_to_swath['183_3H'] = [2,2]
        self.__mFreq_to_swath['183_7H'] = [2,3]
     
    def PCT_Function(self,v,h,val):
        return val[0]*v - val[1]*h
    
    def ReadMatFile(self,region,matFile,valid_freq):
        try:
            mat = scipy.io.loadmat(self.mMatPath+matFile)
        except:
            msg = "Error Reading File: " + str( self.mMatPath+matFile ) 
            print(msg)
            #self.__mLog.error( msg )
       
        swaths = mat["passData"][0][0]
        
        for freq in valid_freq:
            swathList = self.__mFreq_to_swath[freq]
            
            swath_data = swaths[ swathList[0] ]
            lat = swath_data[0][0][1]
            lon = swath_data[0][0][2]
            channel = swath_data[0][0][3]
            
             # Calculating PCT values for specific frequencies
            if freq == '19' or freq=='37' or freq=='91':
                tbs = self.PCT_Function( channel[0], channel[1], swathList[1] )
            else:
                tbs = channel[swathList[1]]
            
            if tbsVal[region][freq][0] >  np.amin(tbs):
                tbsVal[region][freq][0] = np.amin(tbs)

            if tbsVal[region][freq][1] <  np.amax(tbs):
                tbsVal[region][freq][1] = np.amax(tbs)

In [18]:
for region,v1 in valid_matfile_and_best_track.items():
    for year,v2 in v1.items():
        print(region,year)
        for stormNo,stormDict in v2.items():
            for f_,files in stormDict.items():
                rootDirOfMatFile = files[0]
                matFiles = files[1]
                ftbs = FindMaxMinTbs(rootDirOfMatFile)
                for filename,valid_freq in matFiles.items():
                    ftbs.ReadMatFile(region, filename,valid_freq)

ATL 5
ATL 6
ATL 7
ATL 8
ATL 9
ATL 10
ATL 11
ATL 12
ATL 13
ATL 14
ATL 15
ATL 16
ATL 17
ATL 18
ATL 19
CPAC 6
CPAC 9
CPAC 10
CPAC 13
CPAC 14
CPAC 15
CPAC 16
CPAC 18
CPAC 19
EPAC 6
EPAC 7
EPAC 8
EPAC 9
EPAC 10
EPAC 11
EPAC 12
EPAC 13
EPAC 14
EPAC 15
EPAC 16
EPAC 17
EPAC 18
EPAC 19
IO 5
IO 6
IO 7
IO 8
IO 9
IO 10
IO 11
IO 12
IO 13
IO 14
IO 15
IO 16
IO 17
IO 18
IO 19
SHEM 6
SHEM 7
SHEM 8
SHEM 9
SHEM 10
SHEM 11
SHEM 12
SHEM 13
SHEM 14
SHEM 15
SHEM 16
SHEM 17
SHEM 18
SHEM 19
WPAC 5
WPAC 6
WPAC 7
WPAC 8
WPAC 9
WPAC 10
WPAC 11
WPAC 12
WPAC 13
WPAC 14
WPAC 15
WPAC 16
WPAC 17
WPAC 18
WPAC 19


In [11]:
tbsVal

{'19H': [68.83000183105469, 318.2099914550781],
 '19V': [55.2400016784668, 318.3599853515625],
 '19': [-124.24598388671876, 476.7339630126953],
 '22V': [54.7599983215332, 316.6600036621094],
 '37V': [104.0199966430664, 316.3599853515625],
 '37H': [55.04999923706055, 320.0],
 '37': [94.15600357055666, 445.6474998474121],
 '91H': [59.560001373291016, 319.8500061035156],
 '91V': [66.04000091552734, 318.1499938964844],
 '91': [67.13590023040771, 449.1310542755126],
 '150H': [63.86000061035156, 310.5400085449219],
 '183_1H': [61.869998931884766, 310.0400085449219],
 '183_3H': [53.650001525878906, 316.20001220703125],
 '183_7H': [62.7599983215332, 313.2200012207031]}

In [19]:
tbsVal

{'ATL': {'19H': [68.83000183105469, 318.2099914550781],
  '19V': [55.2400016784668, 318.3599853515625],
  '19': [-124.24598388671876, 435.05200500488286],
  '22V': [69.5, 310.9599914550781],
  '37V': [104.0199966430664, 314.32000732421875],
  '37H': [122.80000305175781, 312.30999755859375],
  '37': [94.15600357055666, 445.6474998474121],
  '91H': [64.05999755859375, 318.7099914550781],
  '91V': [66.04000091552734, 309.760009765625],
  '91': [67.13590023040771, 379.3553444290161],
  '150H': [69.12000274658203, 310.5400085449219],
  '183_1H': [61.869998931884766, 310.0400085449219],
  '183_3H': [65.5199966430664, 316.20001220703125],
  '183_7H': [68.5199966430664, 309.92999267578125]},
 'CPAC': {'19H': [99.1500015258789, 286.9599914550781],
  '19V': [178.8699951171875, 290.1600036621094],
  '19': [265.85999755859376, 315.652001953125],
  '22V': [192.6999969482422, 289.2699890136719],
  '37V': [203.47000122070312, 288.3399963378906],
  '37H': [125.95999908447266, 288.0400085449219],
  '37

In [26]:
t = pd.DataFrame(tbsVal)

In [27]:
t

,ATL,CPAC,EPAC,IO,SHEM,WPAC
19H,"[68.83000183105469, 318.2099914550781]","[99.1500015258789, 286.9599914550781]","[94.70999908447266, 302.70001220703125]","[95.19999694824219, 310.4599914550781]","[95.58999633789062, 317.04998779296875]","[92.94000244140625, 298.5799865722656]"
19V,"[55.2400016784668, 318.3599853515625]","[178.8699951171875, 290.1600036621094]","[159.8300018310547, 311.9100036621094]","[179.6199951171875, 317.44000244140625]","[174.8300018310547, 316.8500061035156]","[171.72999572753906, 316.3500061035156]"
19,"[-124.24598388671876, 435.05200500488286]","[265.85999755859376, 315.652001953125]","[191.17600097656248, 420.60197143554694]","[211.07599487304685, 434.57801513671876]","[241.47200622558591, 366.0160186767578]","[184.71798095703122, 476.7339630126953]"
22V,"[69.5, 310.9599914550781]","[192.6999969482422, 289.2699890136719]","[165.08999633789062, 307.70001220703125]","[185.9499969482422, 312.7300109863281]","[184.1699981689453, 316.6600036621094]","[54.7599983215332, 299.3900146484375]"
37V,"[104.0199966430664, 314.32000732421875]","[203.47000122070312, 288.3399963378906]","[183.97000122070312, 311.30999755859375]","[173.72000122070312, 315.0]","[175.61000061035156, 316.3599853515625]","[182.97999572753906, 302.6700134277344]"
37H,"[122.80000305175781, 312.30999755859375]","[125.95999908447266, 288.0400085449219]","[55.04999923706055, 302.2799987792969]","[120.04000091552734, 308.6600036621094]","[124.12999725341797, 303.25]","[121.41999816894531, 320.0]"
37,"[94.15600357055666, 445.6474998474121]","[222.67049484252928, 304.48497924804684]","[186.15499420166017, 349.5574966430664]","[186.8185005187988, 363.62651519775386]","[179.0714942932129, 346.3120094299316]","[187.61200103759762, 370.7715187072754]"
91H,"[64.05999755859375, 318.7099914550781]","[84.66999816894531, 302.489990234375]","[59.560001373291016, 319.8500061035156]","[70.70999908447266, 318.8800048828125]","[64.55000305175781, 318.1300048828125]","[72.06999969482422, 316.8999938964844]"
91V,"[66.04000091552734, 309.760009765625]","[87.70999908447266, 299.4100036621094]","[70.20999908447266, 318.1499938964844]","[72.27999877929688, 317.0]","[66.31999969482422, 311.5899963378906]","[74.19999694824219, 304.30999755859375]"
91,"[67.13590023040771, 379.3553444290161]","[89.99303977203368, 314.8602617492675]","[71.24637702178956, 449.1310542755126]","[73.45906855010986, 338.1336231994628]","[67.64926717376707, 325.324488708496]","[75.79962488555907, 316.91130694580073]"


In [ ]:
plt.clim( tbsVal[self.mReg][self.mSwathMap[i][j]][0] ,tbsVal[self.mReg][self.mSwathMap[i][j]][1])
  
'''
{'19H': [68.83000183105469, 318.2099914550781],
 '19V': [55.2400016784668, 318.3599853515625],
 '19': [-124.24598388671876, 476.7339630126953],
 '22V': [54.7599983215332, 316.6600036621094],
 '37V': [104.0199966430664, 316.3599853515625],
 '37H': [55.04999923706055, 320.0],
 '37': [94.15600357055666, 445.6474998474121],
 '91H': [59.560001373291016, 319.8500061035156],
 '91V': [66.04000091552734, 318.1499938964844],
 '91': [67.13590023040771, 449.1310542755126],
 '150H': [63.86000061035156, 310.5400085449219],
 '183_1H': [61.869998931884766, 310.0400085449219],
 '183_3H': [53.650001525878906, 316.20001220703125],
 '183_7H': [62.7599983215332, 313.2200012207031]}


break

ATL 10 1 F16
20100620T233000_SSMIS_F16.mat
20100621T231800_SSMIS_F16.mat
20100622T115400_SSMIS_F16.mat
20100622T230600_SSMIS_F16.mat
20100623T114100_SSMIS_F16.mat
20100624T003400_SSMIS_F16.mat
20100624T003500_SSMIS_F16.mat
20100624T130900_SSMIS_F16.mat
20100625T002200_SSMIS_F16.mat
20100625T125700_SSMIS_F16.mat
20100626T001000_SSMIS_F16.mat
20100626T124400_SSMIS_F16.mat
20100626T235700_SSMIS_F16.mat
20100627T123200_SSMIS_F16.mat
20100628T012600_SSMIS_F16.mat
20100628T135900_SSMIS_F16.mat
20100629T134600_SSMIS_F16.mat
20100630T010200_SSMIS_F16.mat
20100630T133300_SSMIS_F16.mat
20100701T005000_SSMIS_F16.mat
20100701T132100_SSMIS_F16.mat
20100702T003800_SSMIS_F16.mat
20100702T130900_SSMIS_F16.mat
..\..\CurrentData\Passtimes_and_1C_Data\raw_data\10\ATL\1\passtimes\SSMIS\F16\
'''
a=10